In [2]:
import json
import pandas as pd
import requests
import mysql.connector

# API Request

url = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key=poXVLPnWV8ZufVVMyW2VkoXbjHjPVpF07nbAVPUL"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)

# Load JSON response

data=response.json()

# Convert JSON to DataFrames

categories= data.get("categories", [])  
competitions= data.get("competitions", [])  

# Normalize JSON into DataFrames

df_categories = pd.json_normalize(categories) 
df_competitions = pd.json_normalize(competitions) 

# Replace NaN (Not a Number) values with None

df_categories = df_categories.where(pd.notna(df_categories), None)
df_competitions = df_competitions.where(pd.notna(df_competitions), None)

# Connect to MySQL Database

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="NanDha@12345",
  database="sports_radar_api",
  auth_plugin='mysql_native_password'
)
cursor = mydb.cursor()

# Insert into categories Table (if DataFrame is not empty)

for index, row in df_categories.iterrows():
    sql = "INSERT INTO categories (category_id, category_name) VALUES (%s, %s) ON DUPLICATE KEY UPDATE category_name = VALUES(category_name)"
    values = (row['category_id'], row['category_name'])
    cursor.execute(sql, values)

# Insert into competitions Table (if DataFrame is not empty)

for index, row in df_competitions.iterrows():
    sql = """INSERT INTO competitions (competition_id, competition_name, parent_id, type, gender, category_id)
             VALUES (%s, %s, %s, %s, %s, %s)
             ON DUPLICATE KEY UPDATE 
             competition_name = VALUES(competition_name),
             parent_id = VALUES(parent_id),
             type = VALUES(type),
             gender = VALUES(gender),
             category_id = VALUES(category_id)"""
    
    values = (
    row.get('id', None),
    row.get('name', None),
    row.get('parent_id', None),
    row.get('type', None),
    row.get('gender', None),
    row.get('category.id', None))
    cursor.execute(sql, values)

# Commit Changes & Close Connection

mydb.commit()
cursor.close()
mydb.close()

